<a href="https://colab.research.google.com/github/THS-ST/ocr-models/blob/main/OCR_Models_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **OCR Benchmark Notebook**

This serves to **compare multiple OCR libraries** on your own images. It includes:
- A sample run for the following models: Tesseract (via `pytesseract`), EasyOCR, PaddleOCR, DocTR, TrOCR, and RapidOCR
- A simple evaluation using **WER/CER (via `jiwer`)** *(TBA)*
- Runtime timing per image + per model, and a summary chart *(TBA)*

> **Folder layout:**
>
> - `data/images/` → test images (PNG/JPG/TIFF/PDF-as-images)
> - `data/ground_truth.csv` → optional ground-truth file with columns: `filename,text`  
>   (filenames should match exactly the images in `data/images/`)
>

**Acknowledgement**: Thank you to Ang et al. for allowing us to use their synthetic typewritten prescription dataset for our study.


# **Installations**

In [1]:
!git clone https://github.com/THS-ST/ocr-models

Cloning into 'ocr-models'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 7), reused 23 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 165.21 KiB | 4.13 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:

# --- Install Python packages (run once per environment) ---
%pip -q install paddlepaddle
%pip -q install pytesseract easyocr paddleocr doctr[torch] jiwer opencv-python pillow matplotlib rapidfuzz
%pip -q install python-doctr

# Extra (uncomment as needed):
%pip -q install rapidocr onnxruntime
%pip -q install transformers timm accelerate # for TrOCR (Hugging Face)
%pip -q install mmocr==1.0.1 mmengine==0.10.4 mmdet==3.2.0  # complex; only if you want OpenMMLab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/29

**Tesseract binary is required** for `pytesseract`.

In [3]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# **Initial Imports and Setup**


## System Notes
- GPU is optional but speeds up deep learning models (PaddleOCR, DocTR, Keras-OCR, TrOCR).
- If you see CUDA errors, switch to CPU by installing CPU-only wheels or setting the appropriate flags.


In [4]:

import os, time, json, string, glob, io, math
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Callable, List, Tuple

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

# Evaluation
import jiwer

DATA_DIR = Path("ocr-models/data")
IMAGES_DIR = DATA_DIR / "images"
GT_CSV = DATA_DIR / "ground_truth.csv"

IMAGES_DIR.mkdir(parents=True, exist_ok=True)
print("Expecting images in:", IMAGES_DIR.resolve())
print("Optional ground-truth CSV at:", GT_CSV.resolve())


Expecting images in: /content/ocr-models/data/images
Optional ground-truth CSV at: /content/ocr-models/data/ground_truth.csv


We will load our images with OpenCV (BGR) and return an RGB np.ndarray.

In [5]:
def load_image(path: Path):
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

The following are additional helper functions for loading images and ground truth values.

In [6]:
def list_images(d: Path) -> List[Path]:
    exts = ["*.png", "*.jpg", "*.JPG", "*.jpeg", "*.tif", "*.tiff", "*.bmp"]
    files = []
    for e in exts:
        files.extend(sorted(d.glob(e)))
    return files

def load_ground_truth(gt_csv: Path) -> pd.DataFrame:
    if gt_csv.exists():
        df = pd.read_csv(gt_csv)
        assert {"filename", "text"}.issubset(df.columns), "ground_truth.csv must have columns: filename,text"
        return df
    else:
        return pd.DataFrame(columns=["filename", "text"])

def get_gt_for(fname: str, gt_df: pd.DataFrame) -> str:
    row = gt_df.loc[gt_df["filename"] == fname]
    return "" if row.empty else str(row.iloc[0]["text"])

Next, we define robust text normalization for fair evaluation.

In [7]:
transform = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.Strip(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
])

def compute_metrics(pred: str, ref: str) -> dict:
    if not ref:
        return {"wer": math.nan, "cer": math.nan}
    wer = jiwer.wer(ref, pred, truth_transform=transform, hypothesis_transform=transform)
    cer = jiwer.cer(ref, pred, truth_transform=transform, hypothesis_transform=transform)
    return {"wer": wer, "cer": cer}


Now we can call the functions to load the images and ground truth values.

In [8]:
images = list_images(IMAGES_DIR)
# No ground truth values for now, uncomment as needed
# gt_df = load_ground_truth(GT_CSV)

In [9]:
images

[PosixPath('ocr-models/data/images/2.jpg'),
 PosixPath('ocr-models/data/images/3.jpg'),
 PosixPath('ocr-models/data/images/4.jpg'),
 PosixPath('ocr-models/data/images/5.jpg'),
 PosixPath('ocr-models/data/images/6.jpg'),
 PosixPath('ocr-models/data/images/7.jpg'),
 PosixPath('ocr-models/data/images/8.jpg'),
 PosixPath('ocr-models/data/images/9.jpg'),
 PosixPath('ocr-models/data/images/1.JPG'),
 PosixPath('ocr-models/data/images/10.JPG')]

We will define the following dataclass for our OCR results.

In [10]:
from dataclasses import dataclass

@dataclass
class OCRResult:
    filename: str
    model: str
    pred_text: str
    secs: float

# **1.0: Tesseract**
One of the most widely used OCR engines globally is Tesseract, developed by Google, it utilizes a long short-term memory (LSTM) based neural network for text recognition and has support for over 100 languages (Smith, 2007). It excels at recognizing printed text from high quality images, but performance significantly drops with degraded images or complex layouts. The model also faces challenges with handwriting recognition and images with significant distortions such as noise or skew (Gupta et al., 2021).

We will use the ``pytesseract`` module.

In [11]:
import pytesseract

Now we will define our tesseractOCR function. This will take an ``np.ndarray`` and return an extracted text string.

In [12]:
def ocr_tesseract(img_rgb: np.ndarray) -> str:
    pil_img = Image.fromarray(img_rgb)
    text = pytesseract.image_to_string(pil_img)
    return text.strip()

The results will be stored in this array.

In [13]:
tesseract_results = []

Next, we will call this function for all the images in the directory.

In [14]:
current_model = "tesseract"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_tesseract(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        tesseract_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


Let's turn the array into a dataframe for a better look at our OCR results.

In [15]:
df = pd.DataFrame([r.__dict__ for r in tesseract_results])
df

,filename,model,pred_text,secs
0,2.jpg,tesseract,Fluimucil 600mg #15\nDissoWve 1 tablet in % gl...,0.519314
1,3.jpg,tesseract,Fluimucil 600mg #15\nDissolve 1 tablet in % gl...,0.338595
2,4.jpg,tesseract,1. DecolgeniNeozep\n#15\nSig: Take 1 tablet 3x...,0.425002
3,5.jpg,tesseract,Decolgen/Neozep #15\n1 tablet 3x/day for 5 day...,0.273684
4,6.jpg,tesseract,Decolgen/Neozep #15\n1 tablet three times a da...,0.328644
5,7.jpg,tesseract,1. Betadine Gargle\na\nSig: Gargle 3x/day\n\n2...,0.287530
6,8.jpg,tesseract,Betadine Gargle a\nGargle 3x/day\n\nDiffiam Lo...,0.446513
7,9.jpg,tesseract,Betadine Gargle a\nGargle three times a day\n\...,0.456954
8,1.JPG,tesseract,1. Fluimucil 600mg\n#15\n\nSig: Dissolve 1 tab...,0.970781
9,10.JPG,tesseract,Amoxicillin 500mg cap #21\nSig: take 1 cap 3x ...,0.274231


Since newline characters are printed as is in a dataframe, let's try isolating the text for a better visualization.

In [16]:
for r in tesseract_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[2.jpg]
Fluimucil 600mg #15
DissoWve 1 tablet in % glass water 1x/day.

Immunosin 500mg #21
axiday


[3.jpg]
Fluimucil 600mg #15
Dissolve 1 tablet in % glass water once a day.

Immunosin 500mg #21
Three times a day


[4.jpg]
1. DecolgeniNeozep
#15
Sig: Take 1 tablet 3x/day for 5 days

2. Loratadine 10mg
#21

Sig: Take 1 tablet at breakfast for 14 days


[5.jpg]
Decolgen/Neozep #15
1 tablet 3x/day for 5 days

Loratadine 10mg #21
1 tablet 1x/day at breakfast for 14 days


[6.jpg]
Decolgen/Neozep #15
1 tablet three times a day for 5 days

Loratadine 10mg #21
1 tablet once a day at breakfast for 14 days


[7.jpg]
1. Betadine Gargle
a
Sig: Gargle 3x/day

2. Difflam lozenges
#15
Sig: Every 4 hours


[8.jpg]
Betadine Gargle a
Gargle 3x/day

Diffiam Lozenges #15
1x/4hours


[9.jpg]
Betadine Gargle a
Gargle three times a day

Difflam lozenges #15
Every 4 hours


[1.JPG]
1. Fluimucil 600mg
#15

Sig: Dissolve 1 tablet in % glass water once a
day for 5 days

2. Immunosin 500mg
#21

Sig: Take 1 tab

# **2.0: EasyOCR**


In [17]:
import easyocr

In [18]:
easyocr_reader = easyocr.Reader(['en'], gpu=True if os.environ.get("USE_CUDA", "0") == "1" else False)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [19]:
def ocr_easyocr(img_rgb: np.ndarray) -> str:
    result = easyocr_reader.readtext(img_rgb, detail=0, paragraph=True)
    return "\n".join([r.strip() for r in result if r.strip()])

In [20]:
easyocr_results = []

In [21]:
current_model = "easyocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_easyocr(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        easyocr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


In [22]:
eocr_df = pd.DataFrame([r.__dict__ for r in easyocr_results])
eocr_df

,filename,model,pred_text,secs
0,2.jpg,easyocr,Fluimucil 600mg Dissolve tablet in Yz glass wa...,14.603770
1,3.jpg,easyocr,Fluimucil 600mg Dissolve tablet in Yz glass wa...,13.776708
2,4.jpg,easyocr,Decolgen Neozep #15 Sig: Take tablet 3xlday fo...,12.519234
3,5.jpg,easyocr,Decolgen Neozep tablet 3xlday for 5 days\n#15\...,6.781301
4,6.jpg,easyocr,Decolgen Neozep tablet three times day for 5 d...,6.407636
5,7.jpg,easyocr,Betadine Gargle\nSig: Gargle 3xlday\nDifflam l...,6.792255
6,8.jpg,easyocr,Betadine Gargle Gargle 3xlday\nDifflam Lozenge...,5.805012
7,9.jpg,easyocr,Betadine Gargle Gargle three times day\nDiffla...,6.846431
8,1.JPG,easyocr,Fluimucil 60Omg #15 Sig: Dissolve tablet in Yz...,5.996926
9,10.JPG,easyocr,Amoxicillin 500mg cap #21 Sig: take cap 3x a d...,6.675939


In [23]:
for r in easyocr_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[2.jpg]
Fluimucil 600mg Dissolve tablet in Yz glass water Ixlday:
#15
Immunosin 50Omg 3xlday
#21


[3.jpg]
Fluimucil 600mg Dissolve tablet in Yz glass water once day:
#15
Immunosin 50Omg Three times day
#21


[4.jpg]
Decolgen Neozep #15 Sig: Take tablet 3xlday for 5 days
Loratadine 1Omg #21 Sig: Take tablet at breakfast for 14 days


[5.jpg]
Decolgen Neozep tablet 3xlday for 5 days
#15
Loratadine 1Omg tablet Ixlday at breakfast for 14 days
#21


[6.jpg]
Decolgen Neozep tablet three times day for 5 days
#15
Loratadine 1Omg tablet once day at breakfast for 14 days
#21


[7.jpg]
Betadine Gargle
Sig: Gargle 3xlday
Difflam lozenges #15 Sig: Every 4 hours


[8.jpg]
Betadine Gargle Gargle 3xlday
Difflam Lozenges Ixl4hours
#15


[9.jpg]
Betadine Gargle Gargle three times day
Difflam lozenges Every hours
#15


[1.JPG]
Fluimucil 60Omg #15 Sig: Dissolve tablet in Yz glass water once a day for 5 days
Immunosin 500mg #21 Sig: Take 1 tablet 3xlday for 7 days


[10.JPG]
Amoxicillin 500mg cap #21 Sig:

# **3.0: PaddleOCR**

In [24]:
import paddleocr
from paddleocr import PaddleOCR

In [ ]:
paddle_ocr = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False)

In [26]:
paddle_results = []

In [27]:
current_model = "paddleocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = paddle_ocr.predict(input=img)[0]['rec_texts']
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        paddle_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


In [28]:
pdocr_df = pd.DataFrame([r.__dict__ for r in paddle_results])
pdocr_df

,filename,model,pred_text,secs
0,2.jpg,paddleocr,"[Fluimucil600mg, #15, Dissolve 1tablet in ½ gl...",7.566922
1,3.jpg,paddleocr,"[Fluimucil 600mg, #15, Dissolve 1 tablet in ½ ...",7.577940
2,4.jpg,paddleocr,"[1.Decolgen/Neozep, #15, Sig: Take 1 tablet 3x...",6.548002
3,5.jpg,paddleocr,"[Decolgen/Neozep, #15, 1 tablet 3x/day for 5 d...",7.087564
4,6.jpg,paddleocr,"[Decolgen/Neozep, #15, 1 tablet three times a ...",16.233716
5,7.jpg,paddleocr,"[1. Betadine Gargle, #1, Sig: Gargle 3x/day, 2...",5.005435
6,8.jpg,paddleocr,"[Betadine Gargle, #1, Gargle 3x/day, Diffl am ...",5.889399
7,9.jpg,paddleocr,"[Betadine Gargle, #1, Gargle three times a day...",5.241908
8,1.JPG,paddleocr,"[1. Fluimucil 600mg, #15, Sig: Dissolve 1table...",7.533105
9,10.JPG,paddleocr,"[Amoxicillin 500mg cap #21, Sig: take 1 cap 3x...",4.420268


In [29]:
for r in paddle_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[2.jpg]
['Fluimucil600mg', '#15', 'Dissolve 1tablet in ½ glass water 1x/day.', 'Immunosin 500mg', '#21', '3x/day']


[3.jpg]
['Fluimucil 600mg', '#15', 'Dissolve 1 tablet in ½ glass water once a day', 'Immunosin 500mg', '#21', 'Three times a day']


[4.jpg]
['1.Decolgen/Neozep', '#15', 'Sig: Take 1 tablet 3x/day for 5 days', '2.Loratadine 10mg', '#21', 'Sig: Take 1 tablet at breakfast for 14 days']


[5.jpg]
['Decolgen/Neozep', '#15', '1 tablet 3x/day for 5 days', 'Loratadine 10mg', '#21', '1 tablet 1x/day at breakfast for 14 days']


[6.jpg]
['Decolgen/Neozep', '#15', '1 tablet three times a day for 5 days', 'Loratadine 10mg', '#21', '1 tablet once a day at breakfast for 14 days']


[7.jpg]
['1. Betadine Gargle', '#1', 'Sig: Gargle 3x/day', '2. Dffl am lozenges', '#15', 'Sig: Every 4 hours']


[8.jpg]
['Betadine Gargle', '#1', 'Gargle 3x/day', 'Diffl am Lozenges', '#15', '1x/4hours']


[9.jpg]
['Betadine Gargle', '#1', 'Gargle three times a day', 'Diff am lozenges', '#15', 'Every 4 ho

# **4.0: DocTR**

In [30]:
import doctr
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
from PIL import Image
import pathlib

In [ ]:
doctr_predictor = ocr_predictor(pretrained=True)

In [32]:
def ocr_doctr(img_path: str) -> str:
    # DocTR expects a file or batch; we pass a single image
    doc = DocumentFile.from_images(img_path)
    res = doctr_predictor(doc)
    # extract text blocks in reading order
    lines = []
    for page in res.pages:
        for block in page.blocks:
            for line in block.lines:
                line_text = " ".join([w.value for w in line.words])
                if line_text.strip():
                    lines.append(line_text)
    return "\n".join(lines)

In [33]:
doctr_results = []

In [34]:
current_model = "doctr"

for img_path in images:
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_doctr(str(img_path))
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        doctr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


In [35]:
doctr_df = pd.DataFrame([r.__dict__ for r in doctr_results])
doctr_df

,filename,model,pred_text,secs
0,2.jpg,doctr,Fluimucil 600mg\n#15\nDissolve 1 tablet in % g...,4.658219
1,3.jpg,doctr,Fluimucil 600mg\n#15\nDissolve 1 tablet in % g...,5.690166
2,4.jpg,doctr,1. Decolgen/Neozep\n#15\nSig: Take 1 tablet 3x...,4.993646
3,5.jpg,doctr,Decolgen/Neozep\n#15\n1 tablet 3x/day for 5 da...,5.486886
4,6.jpg,doctr,Decolgen/Neozep\n#15\n1 tablet three times a d...,4.817152
5,7.jpg,doctr,1. Betadine Gargle\n#1\nSig: Gargle 3x/day\n2....,4.556372
6,8.jpg,doctr,Betadine Gargle\n#1\nGargle 3x/day\nDifflam Lo...,4.673026
7,9.jpg,doctr,Betadine Gargle\n#1\nGargle three times a day\...,4.330359
8,1.JPG,doctr,1. Fluimucil 600mg\n#15\nSig: Dissolve 1 table...,6.071387
9,10.JPG,doctr,Amoxicillin 500mg cap #21\nSig: take 1 cap 3x ...,4.262931


In [36]:
for r in doctr_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[2.jpg]
Fluimucil 600mg
#15
Dissolve 1 tablet in % glass water 1X/day.
Immunosin 500mg
#21
3x/day


[3.jpg]
Fluimucil 600mg
#15
Dissolve 1 tablet in % glass water once: a day.
Immunosin 500mg
#21
Three times a day


[4.jpg]
1. Decolgen/Neozep
#15
Sig: Take 1 tablet 3x/day for 5 days
2. Loratadine 10mg
#21
Sig: Take 1 tablet at breakfast for 14 days


[5.jpg]
Decolgen/Neozep
#15
1 tablet 3x/day for 5 days
Loratadine 10mg
#21
1 tablet 1x/day at breakfast for 14 days


[6.jpg]
Decolgen/Neozep
#15
1 tablet three times a day for 5 days
Loratadine 10mg
#21
1 tablet once a day at breaktast for 14 days


[7.jpg]
1. Betadine Gargle
#1
Sig: Gargle 3x/day
2. Difflam lozenges
#15
Sig: Every 4 hours


[8.jpg]
Betadine Gargle
#1
Gargle 3x/day
Difflam Lozenges
#15
1x/4hours


[9.jpg]
Betadine Gargle
#1
Gargle three times a day
Difflam lozenges
#15
Every 4 hours


[1.JPG]
1. Fluimucil 600mg
#15
Sig: Dissolve 1 tablet in % glass water once a
day for 5 days
2. Immunosin 500mg
#21
Sig: Take 1 tablet 3x/d

# **5.0: TrOCR**

In [37]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [ ]:
trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

In [39]:
def ocr_trocr(img_rgb: np.ndarray) -> str:
    pixel_values = trocr_processor(images=Image.fromarray(img_rgb), return_tensors="pt").pixel_values
    outputs = trocr_model.generate(pixel_values)
    return trocr_processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

In [40]:
trocr_results = []

In [41]:
current_model = "trocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_trocr(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        trocr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


In [42]:
trocr_df = pd.DataFrame([r.__dict__ for r in trocr_results])
trocr_df

,filename,model,pred_text,secs
0,2.jpg,trocr,CASHIERIER,5.900672
1,3.jpg,trocr,***,2.524730
2,4.jpg,trocr,CASHIER,3.386468
3,5.jpg,trocr,................,2.461443
4,6.jpg,trocr,:,3.444172
5,7.jpg,trocr,***,2.598216
6,8.jpg,trocr,:,2.477297
7,9.jpg,trocr,:,2.642879
8,1.JPG,trocr,THANK,5.309193
9,10.JPG,trocr,................,2.542025


Notice that the OCR results are extremely inaccurate, this is because TrOCR processes **one line of text at a time**. Because we fed the model the entire image, the results deviate.

Let's try cropping an image and see the results when fed to the model.

In [43]:
crop_dir = IMAGES_DIR / "1.JPG"
test_img = Image.open(crop_dir).convert("RGB")
test_img = test_img.crop((0, 310, 220, 330))
test_img

In [44]:
test_img_np = np.array(test_img)
trocr_cropped_text = ocr_trocr(test_img_np)

print("TrOCR result on cropped image:")
print(trocr_cropped_text)

TrOCR result on cropped image:
1. FLUIMUCIL 600MG


Now the results are accurate to the image.

A possible solution that can allow feeding the model full text images is to create a function that would first split the images into rows of single line text images and perform TrOCR on each split but this would likely be heavy on load and cause long processing times.

# **6.0: RapidOCR**

In [46]:
from rapidocr import RapidOCR

In [47]:
ocr_rapid = RapidOCR()

[INFO] 2025-09-24 11:29:44,074 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-09-24 11:29:44,334 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-09-24 11:29:44,337 [RapidOCR] main.py:55: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-09-24 11:29:44,653 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-09-24 11:29:44,659 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-09-24 11:29:44,662 [RapidOCR] main.py:55: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-09-24 11:29:44,798 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-09-24 11:29:44,950 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

In [48]:
rapid_results = []

In [49]:
current_model = "rapid"

for img_path in images:
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_rapid(str(img_path)).txts
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        rapid_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/2.jpg...


[INFO] 2025-09-24 11:29:50,354 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/resources/fonts/FZYTK.TTF
[INFO] 2025-09-24 11:29:51,927 [RapidOCR] download_file.py:82: Download size: 3.09MB
[INFO] 2025-09-24 11:29:53,144 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/FZYTK.TTF


Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...
Processing: ocr-models/data/images/1.JPG...
Processing: ocr-models/data/images/10.JPG...


In [55]:
rapid_df = pd.DataFrame([r.__dict__ for r in rapid_results])
rapid_df

,filename,model,pred_text,secs
0,2.jpg,rapid,"(Fluimucil600mg, #15, Dissolve1tabletin%glassw...",7.657906
1,3.jpg,rapid,"(Fluimucil600mg, #15, Dissolve1tabletin%glassw...",4.975319
2,4.jpg,rapid,"(1.Decolgen/Neozep, #15, Sig:Take1tablet3x/day...",5.380121
3,5.jpg,rapid,"(Decolgen/Neozep, #15, 1tablet3x/dayfor5days, ...",5.524252
4,6.jpg,rapid,"(Decolgen/Neozep, #15, 1tabletthreetimesadayfo...",4.633637
5,7.jpg,rapid,"(1.BetadineGargle, #1, Sig:Gargle3x/day, 2.Dif...",2.612954
6,8.jpg,rapid,"(Betadine Gargle, #1, Gargle3x/day, Difflam Lo...",2.137873
7,9.jpg,rapid,"(BetadineGargle, #1, Garglethreetimesaday, Dif...",2.368348
8,1.JPG,rapid,"(1.Fluimucil600mg, #15, Sig:Dissolve1tablet in...",3.322222
9,10.JPG,rapid,"(Amoxicillin500mgcap#21, Sig:take 1cap3xadayx1...",2.628629


In [56]:
for r in rapid_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[2.jpg]
('Fluimucil600mg', '#15', 'Dissolve1tabletin%glasswater1x/day.', 'Immunosin500mg', '#21', '3x/day')


[3.jpg]
('Fluimucil600mg', '#15', 'Dissolve1tabletin%glasswateronceaday.', 'Immunosin500mg', '#21', 'Threetimesaday')


[4.jpg]
('1.Decolgen/Neozep', '#15', 'Sig:Take1tablet3x/dayfor5days', '2.Loratadine10mg', '#21', 'Sig:Take1tablet atbreakfastfor14days')


[5.jpg]
('Decolgen/Neozep', '#15', '1tablet3x/dayfor5days', 'Loratadine10mg', '#21', '1tablet1x/dayatbreakfastfor14days')


[6.jpg]
('Decolgen/Neozep', '#15', '1tabletthreetimesadayfor5days', 'Loratadine10mg', '#21', '1tabletonceadayatbreakfastfor14days')


[7.jpg]
('1.BetadineGargle', '#1', 'Sig:Gargle3x/day', '2.Difflam lozenges', '#15', 'Sig:Every4hours')


[8.jpg]
('Betadine Gargle', '#1', 'Gargle3x/day', 'Difflam Lozenges', '#15', '1x/4hours')


[9.jpg]
('BetadineGargle', '#1', 'Garglethreetimesaday', 'Difflam lozenges', '#15', 'Every4hours')


[1.JPG]
('1.Fluimucil600mg', '#15', 'Sig:Dissolve1tablet in% glasswater onc